**Manejo de datos faltantes**
* La diferencia entre los datos que se encuentran en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez son limpios y homogéneos. En particular, muchos conjuntos de datos interesantes tendrán algún grado de datos faltantes. Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar la falta de datos de diferentes maneras.

* En esta sección, discutiremos algunas consideraciones generales sobre los datos faltantes, hablaremos sobre cómo Pandas elige representarlos y demostraremos algunas herramientas integradas de Pandas para manejar datos faltantes en Python. Aquí y a lo largo del libro, nos referiremos a los datos faltantes en general como valores nulos, NaN o NA.

* Ninguno de estos enfoques está libre de compensaciones: el uso de un array de máscara separado requiere la asignación de un array booleano adicional, lo que añade sobrecarga tanto en almacenamiento como en computación. Un valor centinela reduce el rango de valores válidos que pueden representarse y puede requerir lógica adicional (a menudo no optimizada) en la aritmética de CPU y GPU. Valores especiales comunes como NaN no están disponibles para todos los tipos de datos.

**None: Datos faltantes en estilo Python**

* El primer valor centinela utilizado por Pandas es None, un objeto singleton de Python que frecuentemente se utiliza para datos faltantes en código Python. Debido a que None es un objeto de Python, no puede ser utilizado en cualquier array arbitrario de NumPy/Pandas, sino solamente en arrays con tipo de datos 'object' (es decir, arrays de objetos Python):

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1,None,3,4])
vals1

array([1, None, 3, 4], dtype=object)

El tipo de datos dtype=object significa que la mejor representación común de tipo que NumPy pudo inferir para el contenido del array es que son objetos Python. Si bien este tipo de array de objetos es útil para algunos propósitos, cualquier operación en los datos se realizará a nivel de Python, con mucho más sobrecarga que las operaciones típicamente rápidas observadas para arrays con tipos nativos:

In [3]:
for dtype in ['object','int']:
    print("dtype = ",dtype)
    %timeit np.arange(1E6,dtype=dtype).sum()
    print()

dtype =  object
60.9 ms ± 3.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype =  int
2 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



**NaN: Datos numéricos faltantes**

* La otra representación de datos faltantes, NaN (acrónimo de Not a Number), es diferente; es un valor especial de punto flotante reconocido por todos los sistemas que utilizan la representación estándar de punto flotante IEEE:

In [4]:
vals2 = np.array([1,np.nan,3,4])
vals2.dtype

dtype('float64')

Esto significa que los agregados sobre los valores están bien definidos (es decir, no resultan en un error), pero no siempre son útiles:

In [5]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

Numpy proporciona agregados especiales que ignoran los valores faltantes

In [6]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

Ten en cuenta que NaN es específicamente un valor de punto flotante; no existe un valor NaN equivalente para enteros, cadenas u otros tipos de datos.

In [7]:
pd.Series([1,np.nan,2,None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Para tipos que no tienen un valor centinela disponible, Pandas realiza automáticamente una conversión de tipo cuando hay valores NA presentes. Por ejemplo, si establecemos un valor en un array de enteros como np.nan, automáticamente se convertirá a un tipo de punto flotante para acomodar el NA.

In [10]:
x = pd.Series(range(2),dtype=int)
x

0    0
1    1
dtype: int32

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Observa que además de convertir el array de enteros a punto flotante, Pandas convierte automáticamente el None a un valor NaN. (Ten en cuenta que existe una propuesta para añadir un NA nativo para enteros a Pandas en el futuro; hasta la fecha de esta escritura, no ha sido incluido).

**Operando con Valores Nulos**

* Como hemos visto, Pandas trata None y NaN como prácticamente intercambiables para indicar valores faltantes o nulos. Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas. Estos métodos son:

* **isnull():** Genera una máscara booleana que indica los valores faltantes.

* **notnull():**  Contrario a isnull().

* **dropna():** Devuelve una versión filtrada de los datos.

* **fillna():** Devuelve una copia de los datos con los valores faltantes llenados o imputados.

* Concluiremos esta sección con una breve exploración y demostración de estas rutinas.

**Detectando valores nulos**

* Pandas tiene dos metodos para detectar valores nulos, el primero **isnull()** y **notnull()**

In [23]:
data = pd.Series([1,np.nan, 'hello',None])
print(data.isnull())

0    False
1     True
2    False
3     True
dtype: bool


In [27]:
data[data.isnull()]

1     NaN
3    None
dtype: object

In [25]:
data = pd.Series([1,np.nan,'Hello',None])
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

In [28]:
data[data.notnull()]

0        1
2    hello
dtype: object

**Eliminar valores nulos**

* Además del enmascaramiento utilizado anteriormente, existen métodos convenientes como **dropna()** (que elimina valores NA) y **fillna()** (que rellena los valores NA). Para una Serie, el resultado es directo:

In [29]:
data.dropna()

0        1
2    hello
dtype: object

No podemos eliminar valores individuales de un DataFrame; solo podemos eliminar filas completas o columnas completas. Dependiendo de la aplicación, es posible que se desee uno u otro, por lo que dropna() proporciona varias opciones para un DataFrame.


In [34]:
df = pd.DataFrame([[1,np.nan,2],
                  [2,3,5],
                  [np.nan,4,6]]
                  )
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6



Por defecto, dropna() eliminará todas las filas en las que haya al menos un valor nulo presente.

In [33]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Tiene una alternativa para eliminar valores Nan por medio del axis;
* axis = 1 drops all columns contanning a null value:

In [35]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


* Pero esto elimina algunos datos buenos también; es posible que prefieras eliminar filas o columnas que contengan todos los valores NA, o la mayoría de los valores NA. Esto se puede especificar a través de los parámetros how o thresh, los cuales permiten un control preciso sobre la cantidad de nulos que se permiten.

* Por defecto, **how='any'**, de modo que cualquier fila o columna (dependiendo de la palabra clave axis) que contenga al menos un valor nulo será eliminada. 

* También puedes especificar **how='all'**, lo que solo eliminará las filas/columnas que son completamente valores nulos:

In [51]:
df = pd.DataFrame([[1,np.nan,2],
                  [2,3,5],
                  [np.nan,4,6]]
                  )
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [52]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


**how='all'**, lo que solo eliminará las filas/columnas que son completamente valores nulos:

In [50]:
df.dropna(axis='columns',how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Para un control más detallado, el parámetro thresh te permite especificar el número mínimo de valores no nulos que debe tener la fila/columna para que se mantenga:

In [53]:
df.dropna(axis='rows',thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


Rellenar valores nulos

* A veces, en lugar de eliminar valores NA, preferirías reemplazarlos con un valor válido. Este valor podría ser un número único como cero, o podría ser algún tipo de imputación o interpolación a partir de los valores válidos. 

* Podrías hacer esto en el lugar usando el método isnull() como máscara, pero dado que es una operación tan común, Pandas proporciona el método fillna(), que devuelve una copia del array con los valores nulos reemplazados.

In [61]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Remplazando valores por cero

In [56]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [60]:
data.fillna('Faltante')

a         1.0
b    Faltante
c         2.0
d    Faltante
e         3.0
dtype: object

Tambien podemos rellenar valores con el dato anterior

In [62]:
data.fillna(method='ffill')

C:\Users\Mike\AppData\Local\Temp\ipykernel_18040\1844443866.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill')


a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Para los dataframes existe un opcion similar pero se debe especificar el axis

In [63]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [64]:
df.fillna(method='ffill',axis=1)

C:\Users\Mike\AppData\Local\Temp\ipykernel_18040\2272395643.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill',axis=1)


,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
